In [2]:
import pymongo
from pymongo import MongoClient
import pprint
from datetime import datetime
import re
from IPython.display import clear_output

# Replace XXXX with your connection URI from the Atlas UI
client = MongoClient(XXXX)

runtime_pat = re.compile(r'([0-9]+) min')

#we're building up an update document, a document that defines the updates we want to make to an individual document each time through the loop, and then calling update_one 
#to write those updates to the collection changing the document identified by this _ID value each time through the loop.

for movie in client.mflix.movies.find({}).limit(100):
#Limit is a cursor method which will restrict the number of results that we see to, in this case, to the first 100. 
#I could pass 1,000 here. I could pass 365. It just depends on what you need. Limit also returns a cursor.
    
    #fields_to_set and fields_to_unset dictionaries
    fields_to_set = {}
    fields_to_unset = {}

    #In the body of this loop, this is where we are building those dictionaries that we see as the values for set and unset in the update document. 
    for k,v in movie.copy().items():
        if v == "" or v == [""]:
            del movie[k]
            fields_to_unset[k] = ""

    # then we dive into a block of code where I'm splitting string values into arrays.        
    if 'director' in movie:
        fields_to_unset['director'] = ""
        fields_to_set['directors'] = movie['director'].split(", ")
    if 'cast' in movie:
        fields_to_set['cast'] = movie['cast'].split(", ")
    if 'writer' in movie:
        fields_to_unset['writer'] = ""
        fields_to_set['writers'] = movie['writer'].split(", ")
    if 'genre' in movie:
        fields_to_unset['genre'] = ""
        fields_to_set['genres'] = movie['genre'].split(", ")
    if 'language' in movie:
        fields_to_unset['language'] = ""
        fields_to_set['languages'] = movie['language'].split(", ")
    if 'country' in movie:
        fields_to_unset['country'] = ""
        fields_to_set['countries'] = movie['country'].split(", ")
        
    if 'fullplot' in movie:
        fields_to_unset['fullplot'] = ""
        fields_to_set['fullPlot'] = movie['fullplot']
    if 'rating' in movie:
        fields_to_unset['rating'] = ""
        fields_to_set['rated'] = movie['rating']

    #This is the chunk of code that creates the embedded document.     
    imdb = {}
    if 'imdbID' in movie:
        fields_to_unset['imdbID'] = ""
        imdb['id'] = movie['imdbID']
    if 'imdbRating' in movie:
        fields_to_unset['imdbRating'] = ""
        imdb['rating'] = movie['imdbRating']
    if 'imdbVotes' in movie:
        fields_to_unset['imdbVotes'] = ""
        imdb['votes'] = movie['imdbVotes']
    if imdb:
        fields_to_set['imdb'] = imdb
        
    if 'released' in movie:
        fields_to_set['released'] = datetime.strptime(movie['released'],
                                                      "%Y-%m-%d")
    #And here I'm converting string representations of dates to date times using the datetime class.
    if 'lastUpdated' in movie:
        fields_to_set['lastUpdated'] = datetime.strptime(movie['lastUpdated'][0:19],
                                                         "%Y-%m-%d %H:%M:%S")

    # Here, I am changing the data type of runtime. 
    if 'runtime' in movie:
        m = runtime_pat.match(movie['runtime']) 
        if m:
            fields_to_set['runtime'] = int(m.group(1))

    update_doc = {}
    if fields_to_set:
        update_doc['$set'] = fields_to_set
    if fields_to_unset:
        update_doc['$unset'] = fields_to_unset
    pprint.pprint(update_doc)
    
    
    #We're going to be making use of update_one. For update_one, we'll be passing two parameters. This is the most common 
    #way of using one of the update methods in MongoDB. There is a similar method called update_many that allows you 
    #to update multiple documents at once. 
    
    db.movies.update_one({'_id': movie['_id']}, update_doc)
    
    #The _ID field contains a unique identifier for an individual movie in this collection. Here, this filter says, 
    #I'm interested in updating the document with the _ID value equal to the _ID of the document I'm currently processing in this for loop


NameError: name 'XXXX' is not defined